In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import importlib
sys.path.append('../src/')
sys.path.append('../data/')
import models_utils as mu
importlib.reload(mu)
#from data_utils import generate_synthethic, transform_and_normalize, CCCscore
import load_rosmap as lr
importlib.reload(lr)
import data_utils as du
importlib.reload(du)
from sklearn.model_selection import train_test_split
import torch
import os
import numpy as np

In [13]:
## lets use the human brain cortex dataset from the original manuscript
scRNA, _ = lr.load_and_filter_hbc_rosmap(genes_cutoff=3000)

## create pseudobulk for train
xtrain, ytrain = du.generate_synthethic(scRNA, nsamples = 5000)

# ## transform and normalize
xtrain = du.transform_and_normalize(xtrain)

# ## convert to torch
xtrain, = du.convert_to_float_tensors(xtrain)

simulating bulk: 100%|██████████| 5110/5110 [00:24<00:00, 208.64it/s]
/mnt/md0/data/jfuente/digital_cytometry/Sweetwater_package/Sweetwater/examples/../src/data_utils.py:109: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return [torch.tensor(x).float().cuda() if torch.cuda.is_available() else torch.tensor(x).float() for x in args]


In [ ]:
## save it 
modelpath = f'data/weights_{scRNA.shape[1]}_genes.pt'

## load the model
model = mu.SweetWaterAutoEncoder(xtrain.shape[1], num_classes = ytrain.shape[1])
model.load_state_dict(torch.load(os.path.join(modelpath)))
model.eval()

In [ ]:

"""
Compute scores
"""
output = 0 ## gene i
extra = None
deeplift_scores_l = []

for i in tqdm(range(ytrprop.shape[1])):

    if refstr == 'refdynamic':
        input = xtrain[singlect_dict[i],:]
        refdynamic = torch.zeros(size=xtrain[singlect_dict[i],:].shape)

    ## Calculate scores
    deeplift_score = calculate_scores_batch(eval(f'explainer_{explstr}'), input, eval(refstr), i, extra) ## output gene i
    ## sum across samples
    score_values = deeplift_score.cpu().detach().numpy().sum(axis=0)
    deeplift_scores_l.append(score_values)

# visualize score
deeplift_scores_df = pd.DataFrame(deeplift_scores_l).T
deeplift_scores_df.columns = RNAData.celltypes

if explstr == 'lastlayer':
    deeplift_scores_df.index = RNAData.common_genes

# drop unknowns
#ctlist = ['B cells','Monocytes','NK','T CD4','T CD8', 'mDCs', 'pDCs']
#kgenes = remove_ct(RNAData,kgenes, ctlist)

## filter by top k topkdeepliftscore
if typeofgenes == 'topkdeepliftscore':
    topkdeepliftscore = []
    for ct in deeplift_scores_df.columns:
        topkdeeplift_ct = deeplift_scores_df.loc[:,ct].sort_values(ascending=False).head(ngenes_celltype).index.tolist()
        topkdeepliftscore += topkdeeplift_ct

kgenes = eval(typeofgenes) #+ bottomkgenes
deeplift_scores_df_kgenes = deeplift_scores_df.loc[kgenes, :]
deeplift_scores_df_kgenes.to_csv(os.path.join('interpretability', name.lower(), 'sweetwater', 'lastlayer','topkdeepliftscore',f'score_genes_top{ngenes_celltype}.tsv'),sep='\t')

##define filename
filename = f'corrmat_{xtrain.shape[1]}_sumscores_{refstr}_top{ngenes_celltype}genes'

## plot complex heatmap
plot_complexheatmap(rename_duplicates(deeplift_scores_df_kgenes))